In [1]:
#import
import requests
from bs4 import BeautifulSoup
import itertools
from dataclasses import dataclass
import datetime as dt


## Скачать основную информацию 

In [54]:
# helpers


# Словарь для преобразования сокращенных названий дней недели
days_translation = {
    'Пн': 'Mon',
    'Вт': 'Tue',
    'Ср': 'Wed',
    'Чт': 'Thu',
    'Пт': 'Fri',
    'Сб': 'Sat',
    'Вс': 'Sun'
}

# Словарь для преобразования сокращенных названий месяцев
months_translation = {
    'Янв': 'Jan',
    'Фев': 'Feb',
    'Мар': 'Mar',
    'Апр': 'Apr',
    'Май': 'May',
    'Июн': 'Jun',
    'Июл': 'Jul',
    'Авг': 'Aug',
    'Сен': 'Sep',
    'Окт': 'Oct',
    'Ноя': 'Nov',
    'Дек': 'Dec'
}

def parse_date(date_str):
    # Разделяем строку на части
    parts = date_str.split()

    # Переводим день недели и месяц
    translated_day = days_translation.get(parts[0])
    translated_month = months_translation.get(parts[1])

    # Проверяем, были ли переведены день и месяц
    if translated_day is None or translated_month is None:
        raise ValueError("Некорректный формат даты или неизвестный день/месяц.")

    # Создаем новую строку с переведенным днем недели и месяцем
    translated_date_str = f"{translated_day} {translated_month} {parts[2]} {parts[3]}"

    # Определяем формат для парсинга
    date_format = '%a %b %d %Y'

    # Преобразуем строку в объект datetime
    return dt.datetime.strptime(translated_date_str, date_format)

### собираем ссылки

In [88]:
def get_ids_from_one_page(page=''):
    """
    page: id страниц - 12(2), 24(3), 36(4) и 
    """
    url = f'https://quest-book.ru/online/list/where/?&s={page}'
    response = requests.get(url)
    #print(url)
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        row = soup.find_all(class_='game-list')[0].find_all('div', class_='row')[0]
        game_ids = []
        game_info_divs = row.find_all('div', class_='game_info')
        for game_info in game_info_divs:
            game_id = game_info.get('data-game-id')
            if game_id:
                game_ids.append(game_id)
        return game_ids
    else:
        print(f"Ошибка: {response.status_code}")
        return []

def get_ids_by_pages():
    ids = []
    current_page_ids = get_ids_from_one_page()
    i = 12
    while current_page_ids != []:
        ids.append(current_page_ids)
        current_page_ids = get_ids_from_one_page(page=f'{i}')
        #print(current_page_ids)
        i += 12
    ids = list(itertools.chain.from_iterable(ids))
    return ids

def get_story_info(id):
    try:
        url = f'https://quest-book.ru/online/view/game{id}'
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content.encode('ISO-8859-1'), 'html.parser')

            author_info = {}

            author_cell = soup.find('td', text=lambda x: 'Автор' in x)

            # Если ячейка найдена, получить родительский элемент <tr>
            if author_cell:
                author_cell = author_cell.find_next('td')
                author_link = author_cell.find('a')
                author_name = author_link.text.strip()
                author_info['Имя'] = author_name
            print('Имя',author_info['Имя'])
                
            # Извлечение краткого описания
            description_row = soup.find(text="Краткое описание")
            if description_row:
                description_cell = description_row.find_next('td')
                author_info['Краткое описание'] = description_cell.text.strip()
            #print('Краткое описание',author_info.get('Краткое описание'))

            # Извлечение категорий
            categories_row = soup.find(text=lambda x: 'Категории' in x)
            if categories_row:
                categories_cell = categories_row.find_next('td')
                categories = []
                for div in categories_cell.find_all('div'):
                    genre = div.find('b').text.strip()
                    if genre == 'Жанр':
                        gs = div.find_all('a')
                        for g in gs:
                            categories.append(g.text.strip())
                author_info['Категории'] = categories
            #print('Категории',author_info.get('Категории'))


            # Извлечение последнего обновления
            update_row = soup.find(text=lambda x: 'Последнее обновление' in x)
            if update_row:
                update_cell = update_row.find_next('td')
                last_update_str = update_cell.text.strip().split(',')[0] + update_cell.text.strip().split(',')[1]
                last_update = parse_date(last_update_str)
                author_info['Последнее обновление'] = last_update
            #print('Последнее обновление',author_info.get('Последнее обновление'))

            # Извлечение информации о саундтреке
            soundtrack_row = soup.find(text=lambda x: 'Саундтрек' in x)
            if soundtrack_row:
                soundtrack_cell = soundtrack_row.find_next('td')
                author_info['Саундтрек'] = soundtrack_cell.text.strip()
            #print('Саундтрек',author_info.get('Саундтрек'))

            activity_row = soup.find(text=lambda x: 'Активность' in x)
            if activity_row:
                activity_cell = activity_row.find_next('td')
                activity_data = activity_cell.find('div').text.strip().split(': ')
                
                # Извлечение запусков за месяц и всего
                launches_last_month = int(activity_data[1].split('\r')[0])
                total_launches = int(activity_data[2].split('\r')[0])
                
                author_info['Запусков за месяц'] = launches_last_month
                author_info['Всего запусков'] = total_launches
            #print('Запусков за месяц',author_info.get('Запусков за месяц'))

            size_row = soup.find(text=lambda x: 'Размер' in x)
            if size_row:
                size_cell = size_row.find_next('td').text.strip().split(': ')
                #size_data = size_cell.find(text=True).strip().split('</br>')
                size_data = size_cell 
                
                # Извлечение количества параграфов и концовок
                paragraphs_count = int(size_data[1].split('\r')[0])
                endings_count = int(size_data[2].split('\r')[0])
                
                author_info['Количество параграфов'] = paragraphs_count
                author_info['Количество концовок'] = endings_count

            id = id
            name = soup.find('h2', class_='mt-1').text
            author = author_info.get('Имя')
            date_last_version = author_info.get('Последнее обновление')
            short_description = author_info.get('Краткое описание')
            last_month_readed = author_info.get('Запусков за месяц')
            all_readed = author_info.get('Всего запусков')
            likes = soup.find('span', class_='label label-success').find('b').text
            dislikes = soup.find('span', class_='label label-warning').find('b').text
            part_count = author_info.get('Количество параграфов') 
            end_count = author_info.get('Количество концовок')
            music = author_info.get('Саундтрек')
            genres = author_info.get('Категории')
            return Story(id=id,
                        name=name, 
                        author=author,
                        date_last_version=date_last_version,
                        short_description=short_description,
                        last_month_readed=last_month_readed,
                        all_readed=all_readed,
                        likes=likes,
                        dislikes=dislikes,
                        part_count=part_count,
                        end_count=end_count,
                        music=music,
                        genres=genres)
        else:
            print(f"Ошибка: {response.status_code}")
            return None
    except Exception as e:
        print(e)
        return None

@dataclass    
class Story:
    id:int
    name:str
    author:str
    date_last_version: dt.datetime
    short_description:str 
    last_month_readed:int
    all_readed:int
    likes:int
    dislikes:int
    part_count:int
    end_count:int
    music:str
    genres: list[str]

    
    
    

In [3]:
story_ids =  get_ids_by_pages()
print(len(story_ids))

641


In [4]:
len(story_ids)

641

In [89]:
stories=[]
for id in story_ids:
    print(id)
    story =get_story_info(id)
    print(story)
    stories.append(story)
    #print(story['name'])

17423


C:\Users\User\AppData\Local\Temp\ipykernel_11760\1388852564.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_cell = soup.find('td', text=lambda x: 'Автор' in x)
C:\Users\User\AppData\Local\Temp\ipykernel_11760\1388852564.py:56: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description_row = soup.find(text="Краткое описание")
C:\Users\User\AppData\Local\Temp\ipykernel_11760\1388852564.py:63: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  categories_row = soup.find(text=lambda x: 'Категории' in x)
C:\Users\User\AppData\Local\Temp\ipykernel_11760\1388852564.py:78: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  update_row = soup.find(text=lambda x: 'Последнее обновление' in x)
C:\Users\User\AppData\Local\Temp\ipykernel_11760\1388852564.py:87: DeprecationWarning

Имя Da_Tenshi
Запусков за месяц 42
['Параграфов', '62\r\n\t\t\t\t\t\t\t\t\tКонцовок', '14']
Story(id='17423', name='Визит в дом 16', author='Da_Tenshi', date_last_version=datetime.datetime(2025, 1, 18, 0, 0), short_description='детектив, криминал, конец 19 века', last_month_readed=42, all_readed=42, likes='1', dislikes='1', part_count=62, end_count=14, music='Нет', genres=['детектив'])
17415
Имя Smurfik 228
Запусков за месяц 22
['Параграфов', '26\r\n\t\t\t\t\t\t\t\t\tКонцовок', '10']
Story(id='17415', name='Память', author='Smurfik 228', date_last_version=datetime.datetime(2025, 1, 15, 0, 0), short_description='вы всё забыли и должны по кусочкам восстанавливать себе память', last_month_readed=22, all_readed=22, likes='0', dislikes='3', part_count=26, end_count=10, music='Нет', genres=['поиск', 'повседневность'])
17403
Имя Pete Pr
Запусков за месяц 141
['Параграфов', '32\r\n\t\t\t\t\t\t\t\t\tКонцовок', '5']
Story(id='17403', name='Спасение нерядовой ели', author='Pete Pr', date_last_ver

KeyboardInterrupt: 